In [1]:
!pip install keras-tuner

     -------------------------------------- 176.1/176.1 kB 1.3 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.12.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 2s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\afrah\AppData\Local\Temp\ipykernel_22076\556418634.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 15m 44s]
val_accuracy: 0.8958333134651184

Best val_accuracy So Far: 0.9100000262260437
Total elapsed time: 01h 01m 08s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 64)        1664      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 48)        76848     
                                                                 
 flatten (Flatten)           (None, 19200)             0         
                                                                 
 dense (Dense)               (None, 112)               2150512   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 2,230,154
Trainable params: 2,230,154
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 293s 172ms/step - loss: 0.1678 - accuracy: 0.9356 - val_loss: 0.2638 - val_accuracy: 0.9083
Epoch 5/10
1688/1688 [==============================] - 220s 130ms/step - loss: 0.1351 - accuracy: 0.9486 - val_loss: 0.2749 - val_accuracy: 0.9113
Epoch 6/10
1688/1688 [==============================] - 215s 127ms/step - loss: 0.1017 - accuracy: 0.9626 - val_loss: 0.2628 - val_accuracy: 0.9160
Epoch 7/10
1688/1688 [==============================] - 217s 129ms/step - loss: 0.0809 - accuracy: 0.9702 - val_loss: 0.3475 - val_accuracy: 0.9127
Epoch 8/10
1688/1688 [==============================] - 218s 129ms/step - loss: 0.0643 - accuracy: 0.9763 - val_loss: 0.3750 - val_accuracy: 0.9142
Epoch 9/10
1688/1688 [==============================] - 213s 126ms/step - loss: 0.0509 - accuracy: 0.9811 - val_loss: 0.3736 - val_accuracy: 0.9123
Epoch 10/10
1688/1688 [==============================] - 211s 125ms/step - loss: 0.0411 - accuracy: 0.9858 - val